# Imports

In [1]:
%pip uninstall -y scikit-learn xgboost imblearn
%pip install scikit-learn==1.3.2 xgboost==2.0.3 imblearn==0.12.0

import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# df_trn = pd.read_csv("../data/guiDataCleanInfill0Treino.csv")
# df_tst = pd.read_csv("../data/guiDataCleanInfill0Teste.csv")

# df_trn = pd.read_csv("../data/guiDataCleanKNNTreino.csv")
# df_tst = pd.read_csv("../data/guiDataCleanKNNTeste.csv")

df_trn = pd.read_csv("../data/guiDataCleanMaxDataTreino.csv")
df_tst = pd.read_csv("../data/guiDataCleanMaxDataTeste.csv")

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: xgboost 3.0.2
Uninstalling xgboost-3.0.2:
  Successfully uninstalled xgboost-3.0.2
Note: you may need to restart the kernel to use updated packages.
  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
ERROR: Could not find a version that satisfies the requirement imblearn==0.12.0 (from versions: 0.0)
ERROR: No matching distribution found for imblearn==0.12.0
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Dividir X e y
X_train, y_train = df_trn.drop('target', axis=1), df_trn['target']
X_test, y_test = df_tst.drop('target', axis=1), df_tst['target']

# SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Scaling
scaler = StandardScaler()
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# usar RandomizedSearchCV para encontrar os melhores hiperparâmetros
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [2, 3, 4, 5],
    'min_child_weight': [1, 3, 5, 10],
    'gamma': [0, 0.1, 0.5, 1],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 5, 10]
}
cv = 5  # número de folds para validação cruzada
xgb_clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',
    cv=cv,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_resampled_scaled, y_train_resampled)

# Previsões e resultados
y_pred = random_search.predict(X_test_scaled)

print("Melhores parâmetros encontrados:", random_search.best_params_)

AttributeError: 'super' object has no attribute '__sklearn_tags__'